# Train DL

> A collection of deep learning tools via Fastai

## Setup

In [ ]:
#| default_exp dl

In [ ]:
#| hide
import sys
sys.path.append("/notebooks/katlas")
from nbdev.showdoc import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
from fastbook import *
import fastcore.all as fc,torch.nn.init as init
from fastai.callback.training import GradientClip
from torch.utils.data import WeightedRandomSampler

# katlas
from katlas.core import Data
from katlas.feature import *
from katlas.train import *

# sklearn
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr,pearsonr

## Utils

In [ ]:
#| export
def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
seed_everything()

In [ ]:
#| export
def_device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def_device

'cpu'

## Load Data

In [ ]:
# read training data
df = pd.read_parquet('train_data/combine_t5_kd.parquet').reset_index()

# read data contains info for split
info_df = Data.get_kinase_info().query('pseudo!="1"') # get non-pseudo kinase

# merge info with training data
info = df[['kinase']].merge(info_df)
info.head()

# splits
splits = get_splits(info,stratified='group')
split0 = splits[0]


# column name of feature and target
feat_col = df.columns[df.columns.str.startswith('T5_')]
target_col = df.columns[~df.columns.isin(feat_col)][1:]

StratifiedKFold(n_splits=5, random_state=123, shuffle=True)
# kinase group in train set: 9
# kinase group in test set: 9
---------------------------
# kinase in train set: 312
---------------------------
# kinase in test set: 78
---------------------------
test set: ['EPHA3' 'FES' 'FLT3' 'FYN' 'EPHB1' 'EPHB3' 'FER' 'EPHB4' 'FLT4' 'FGFR1' 'EPHA5' 'TEK' 'DDR2' 'ZAP70' 'LIMK1' 'ULK3' 'JAK1' 'WEE1' 'TESK1' 'MAP2K3' 'AMPKA2' 'ATM' 'CAMK1D' 'CAMK2D' 'CAMK4' 'CAMKK1'
 'CK1D' 'CK1E' 'DYRK2' 'DYRK4' 'HGK' 'IKKE' 'JNK2' 'JNK3' 'KHS1' 'MAPKAPK5' 'MEK2' 'MSK2' 'NDR1' 'NEK6' 'NEK9' 'NIM1' 'NLK' 'OSR1' 'P38A' 'P38B' 'P90RSK' 'PAK1' 'PERK' 'PKCH' 'PKCI' 'PKN1' 'ROCK2'
 'RSK2' 'SIK' 'STLK3' 'TAK1' 'TSSK1' 'ALPHAK3' 'BMPR2' 'CDK10' 'CDK13' 'CDK14' 'CDKL5' 'GCN2' 'GRK4' 'IRE1' 'KHS2' 'MASTL' 'MLK4' 'MNK1' 'MRCKA' 'PRPK' 'QSK' 'SMMLCK' 'SSTK' 'ULK2' 'VRK1']


## Dataset

In [ ]:
#| export
class GeneralDataset:
    def __init__(self, 
                 df, # a dataframe of values
                 feat_col, # feature columns
                 target_col=None # Will return test set for prediction if target col is None
                ):
        "A general dataset that can be applied to any dataframe"
        
        self.test = False if target_col is not None else True
        
        self.X = df[feat_col].values 
        self.y = df[target_col].values if not self.test else None
        
        self.len = df.shape[0]

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        X = torch.Tensor(self.X[index])
        if self.test:
            return X
        else:
            y = torch.Tensor(self.y[index])
            return X, y

In [ ]:
# dataset
ds = GeneralDataset(df,feat_col,target_col)

In [ ]:
len(ds)

390

In [ ]:
dl = DataLoader(ds, batch_size=64, shuffle=True)

In [ ]:
#| export
def get_sampler(info,col):
    
    "For imbalanced data, get higher weights for less-represented samples"
    
    # get value counts
    group_counts = info[col].value_counts()
    
    # to reduce the difference through log
    # group_counts = group_counts.apply(lambda x: np.log(x+1.01))
    
    weights = 1. / group_counts[info[col]]

    sample_weights = torch.from_numpy(weights.to_numpy())
    sample_weights = torch.clamp_min(sample_weights,0.01)

    sampler = WeightedRandomSampler(sample_weights, len(sample_weights),replacement=True)
    
    return sampler

In [ ]:
sampler = get_sampler(info,'subfamily')

In [ ]:
# dataloader
dl = DataLoader(ds, batch_size=64, sampler=sampler)

In [ ]:
xb,yb = next(iter(dl))

xb.shape,yb.shape

(torch.Size([64, 1024]), torch.Size([64, 210]))

## Models

### MLP

In [ ]:
#| export
def MLP_1(num_features, 
          num_targets,
          hidden_units = [512, 218],
          dp = 0.2):
    
    # Start with the first layer from num_features to the first hidden layer
    layers = [
        nn.Linear(num_features, hidden_units[0]),
        nn.BatchNorm1d(hidden_units[0]),
        nn.Dropout(dp),
        nn.PReLU()
    ]
    
    # Loop over hidden units to create intermediate layers
    for i in range(len(hidden_units) - 1):
        layers.extend([
            nn.Linear(hidden_units[i], hidden_units[i+1]),
            nn.BatchNorm1d(hidden_units[i+1]),
            nn.Dropout(dp),
            nn.PReLU()
        ])
    
    # Add the output layer
    layers.append(nn.Linear(hidden_units[-1], num_targets))
    
    model = nn.Sequential(*layers)
    
    return model

In [ ]:
n_feature = len(feat_col)
n_target = len(target_col)

In [ ]:
model = MLP_1(n_feature, n_target)

In [ ]:
model(xb)

tensor([[-0.1115, -0.3755, -0.3818,  ..., -0.1483, -0.0387, -0.1111],
        [ 0.8555,  0.9352, -0.9642,  ..., -0.4723,  0.7757, -0.0121],
        [ 0.3422,  0.3537, -0.1441,  ...,  0.5467, -0.4535,  0.2103],
        ...,
        [-0.4287,  0.6751,  0.1797,  ...,  0.0192,  0.0692, -0.0573],
        [-0.0206, -0.1953,  0.7445,  ..., -0.2206, -0.1188,  0.4579],
        [ 0.2342, -0.0243,  0.4630,  ...,  0.8393,  0.5747, -0.6881]], grad_fn=<AddmmBackward0>)

### CNN1D

***Version 1***

In [ ]:
#| export
class CNN1D_1(Module):
    
    def __init__(self, 
                 num_features, # this does not matter, just for format
                 num_targets):

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3, dilation=1, padding=1, stride=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=3, out_channels=8, kernel_size=3, dilation=1, padding=1, stride=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.flatten = Flatten()
        self.fc1 = nn.Linear(in_features = int(8 * num_features/4), out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=num_targets)

    def forward(self, x):
        x = x.unsqueeze(1) # need shape (bs, 1, num_features) for CNN
        x = self.pool1(nn.functional.relu(self.conv1(x)))
        x = self.pool2(nn.functional.relu(self.conv2(x)))
        # x = torch.flatten(x, 1)
        x = self.flatten(x)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
model = CNN1D_1(n_feature, n_target)

In [ ]:
model(xb)

tensor([[ 0.0193,  0.0690,  0.0138,  ..., -0.0428, -0.0026,  0.0840],
        [ 0.0203,  0.0693,  0.0136,  ..., -0.0422, -0.0023,  0.0846],
        [ 0.0198,  0.0703,  0.0148,  ..., -0.0424, -0.0029,  0.0839],
        ...,
        [ 0.0197,  0.0694,  0.0147,  ..., -0.0429, -0.0019,  0.0841],
        [ 0.0193,  0.0687,  0.0146,  ..., -0.0429, -0.0017,  0.0843],
        [ 0.0191,  0.0692,  0.0148,  ..., -0.0425, -0.0028,  0.0834]], grad_fn=<AddmmBackward0>)

***Version 2***

In [ ]:
#| export
def init_weights(m, leaky=0.):
    "Initiate any Conv layer with Kaiming norm."
    if isinstance(m, (nn.Conv1d,nn.Conv2d,nn.Conv3d)): init.kaiming_normal_(m.weight, a=leaky)

In [ ]:
#| export
def lin_wn(ni,nf,dp=0.1,act=nn.SiLU):
    "Weight norm of linear."
    layers =  nn.Sequential(
            nn.BatchNorm1d(ni),
            nn.Dropout(dp),
            nn.utils.weight_norm(nn.Linear(ni, nf)) )
    if act: layers.append(act())
    return layers

In [ ]:
#| export
def conv_wn(ni, nf, ks=3, stride=1, padding=1, dp=0.1,act=nn.ReLU):
    "Weight norm of conv."
    layers =  nn.Sequential(
        nn.BatchNorm1d(ni),
        nn.Dropout(dp),
        nn.utils.weight_norm(nn.Conv1d(ni, nf, ks, stride, padding)) )
    if act: layers.append(act())
    return layers

In [ ]:
#| export
class CNN1D_2(nn.Module):
    
    def __init__(self, ni, nf, amp_scale = 16):
        super().__init__()

        cha_1,cha_2,cha_3 = 256,512,512
        hidden_size = cha_1*amp_scale

        cha_po_1 = hidden_size//(cha_1*2)
        cha_po_2 = (hidden_size//(cha_1*4)) * cha_3
        
        self.lin = lin_wn(ni,hidden_size)
        
        # bs, 256, 16
        self.view = View(-1,cha_1,amp_scale)
        
        self.conv1 = nn.Sequential(
            conv_wn(cha_1, cha_2, ks=5, stride=1, padding=2, dp=0.1),
            nn.AdaptiveAvgPool1d(output_size = cha_po_1),
            conv_wn(cha_2, cha_2, ks=3, stride=1, padding=1, dp=0.1))
        
        self.conv2 = nn.Sequential(
            conv_wn(cha_2, cha_2, ks=3, stride=1, padding=1, dp=0.3),
            conv_wn(cha_2, cha_3, ks=5, stride=1, padding=2, dp=0.2))
        
        self.head = nn.Sequential(
            nn.MaxPool1d(kernel_size=4, stride=2, padding=1),
            nn.Flatten(),
            lin_wn(cha_po_2,nf,act=None) )


    def forward(self, x):
        # amplify features to 4096
        x = self.lin(x)
        
        # reshape to bs,256,16 for conv1d
        x = self.view(x) 

        x = self.conv1(x)
        
        x_s = x  # for skip connection (multiply)
        x = self.conv2(x)
        x = x * x_s

        # Final block
        x = self.head(x)

        return x

In [ ]:
model = CNN1D_2(n_feature,n_target).apply(init_weights)

In [ ]:
model(xb)

tensor([[-0.5740,  0.0151, -0.0819,  ...,  0.2636,  0.3405, -0.1404],
        [-0.6800,  0.5530, -0.0958,  ..., -0.3752, -0.6124,  0.7171],
        [ 0.4427, -0.3204, -0.3243,  ..., -0.2290,  0.1070,  0.1504],
        ...,
        [-0.3660, -0.2667, -0.6036,  ..., -0.3130,  0.5462, -0.0055],
        [ 0.4511,  0.6824,  0.8659,  ..., -0.0171,  0.2362, -0.3475],
        [-0.0746, -0.1699,  0.6895,  ...,  1.1522, -0.3472,  0.6422]], grad_fn=<AddmmBackward0>)

## DL Trainer

In [ ]:
#| export
def train_dl(df, 
            feat_col, 
            target_col,
            split, # tuple of numpy array for split index
            model_func, # function to get pytorch model
             n_epoch = 4, # number of epochs
             bs = 32, # batch size
             lr = 1e-2, # will be useless if lr_find is True
            loss = mse, # loss function
            save = None, # models/{save}.pth
             sampler = None,
             lr_find=False, # if true, will use lr from lr_find
              ):
    "A DL trainer."
    
    train = df.loc[split[0]]
    valid = df.loc[split[1]]
    
    train_ds = GeneralDataset(train, feat_col, target_col)
    valid_ds = GeneralDataset(valid, feat_col, target_col)
    
    n_workers = fc.defaults.cpus

    if sampler is not None:
        
        train_dl = DataLoader(train_ds, batch_size=bs, sampler=sampler,num_workers=n_workers)
        valid_dl = DataLoader(valid_ds, batch_size=bs, sampler=sampler,num_workers=n_workers)
        
        dls = DataLoaders(train_dl, valid_dl)
        
    else:
        
        dls = DataLoaders.from_dsets(train_ds, valid_ds, bs=bs, num_workers=n_workers)
    
    model = model_func()
    
    learn = Learner(dls.to(def_device), model.to(def_device), loss, 
                    metrics= [PearsonCorrCoef(),SpearmanCorrCoef()],
                    cbs = [GradientClip(1.0)] # prevent overfitting
                   )
    
    if lr_find:
        # get learning rate
        lr = learn.lr_find()
        plt.show()
        plt.close()
        print(lr)

        
    print('lr in training is', lr)
    learn.fit_one_cycle(n_epoch,lr) #cbs = [SaveModelCallback(fname = 'best')] # save best model
    
    if save is not None:
        learn.save(save)
        
    pred,target = learn.get_preds()
    
    pred = pd.DataFrame(pred.detach().cpu().numpy(),index=valid.index,columns=target_col)
    target = pd.DataFrame(target.detach().cpu().numpy(),index=valid.index,columns=target_col)
    
    return target, pred

In [ ]:
def get_model():
    return CNN1D_2(n_feature, n_target)

In [ ]:
target, pred = train_dl(df, 
                        feat_col, 
                        target_col,
                        split0, 
                        get_model,
                        n_epoch=1,
                        lr = 1e-2,
                        save = 'test')

lr in training is 0.01


epoch,train_loss,valid_loss,pearsonr,spearmanr,time
0,2.194746,2.176958,-0.104578,-0.053141,00:04


In [ ]:
score_each(target,pred)

overall MSE: 2.1770
Average Pearson: 0.2149 


(2.176958,
 0.21488270397776432,
       Pearson
 3   -0.442855
 8   -0.490345
 10  -0.401885
 19  -0.428557
 24  -0.383956
 ..        ...
 359 -0.127000
 361  0.005761
 366  0.095977
 367  0.335805
 373 -0.230842
 
 [78 rows x 1 columns])

## DL CV

In [ ]:
#| export
@fc.delegates(train_dl)
def train_dl_cv(df, 
                feat_col, 
                target_col, 
                splits, # list of tuples
                model_func, # functions like lambda x: return MLP_1(num_feat, num_target)
                save:str=None,
                **kwargs
                ):
    
    OOF = []
    metrics = []
    
    for fold,split in enumerate(splits):

        print(f'------fold{fold}------')
        
        
        fname=None
        # save best model for each fold
        if save is not None:
            fname = f'{save}_fold{fold}'
        
        # train model
        target, pred = train_dl(df,feat_col,target_col, split, model_func ,save=fname,**kwargs)

        #------------get scores--------------
        # get score metrics
        mse, pearson_avg, _ = score_each(target,pred)
        
        # store metrics in a dictionary for the current fold
        fold_metrics = {
            'fold': fold,
            'mse': mse,
            'pearson_avg': pearson_avg
        }
        metrics.append(fold_metrics)

        OOF.append(pred)
        

    # Concatenate OOF from each fold to a new dataframe
    oof = pd.concat(OOF).sort_index()
    
    # Get metrics into a dataframe
    metrics = pd.DataFrame(metrics)
    
    return oof, metrics

In [ ]:
def get_model():
    return CNN1D_2(n_feature, n_target)

In [ ]:
oof,metrics = train_dl_cv(df,feat_col,target_col,splits,get_model,n_epoch=1,lr=3e-3)

------fold0------
lr in training is 0.003


epoch,train_loss,valid_loss,pearsonr,spearmanr,time
0,1.165076,0.997911,0.091948,0.058285,00:01


overall MSE: 0.9979
Average Pearson: 0.1634 
------fold1------
lr in training is 0.003


epoch,train_loss,valid_loss,pearsonr,spearmanr,time
0,1.180757,0.992539,0.102852,0.084205,00:01


overall MSE: 0.9925
Average Pearson: 0.1617 
------fold2------
lr in training is 0.003


...

overall MSE: 0.9872
Average Pearson: 0.2364 
------fold3------
lr in training is 0.003


epoch,train_loss,valid_loss,pearsonr,spearmanr,time
0,1.184666,1.001829,0.077155,0.047876,00:01


overall MSE: 1.0018
Average Pearson: 0.1415 
------fold4------
lr in training is 0.003


epoch,train_loss,valid_loss,pearsonr,spearmanr,time
0,1.178444,0.992547,0.109969,0.100576,00:01


overall MSE: 0.9925
Average Pearson: 0.2014 


In [ ]:
metrics

,fold,mse,pearson_avg
0,0,0.997911,0.163423
1,1,0.992539,0.161654
2,2,0.987170,0.236363
3,3,1.001829,0.141464
4,4,0.992547,0.201375


In [ ]:
metrics.pearson_avg.mean()

0.18085578818910147

In [ ]:
target = df[target_col]
_,_,corr = score_each(target,oof)

overall MSE: 0.9944
Average Pearson: 0.1809 


In [ ]:
corr

,Pearson
0,-0.183429
1,-0.178564
2,-0.225202
3,-0.117838
4,-0.153463
...,...
385,0.109792
386,0.269238
387,0.079601
388,0.063310


## DL Predict

In [ ]:
#| export
def predict_dl(df, 
               feat_col, 
               target_col,
               model, # model architecture
               model_pth, # only name, not with .pth
              ):
    
    "Predict dataframe given a deep learning model"
    
    test_dset = GeneralDataset(df,feat_col)
    test_dl = DataLoader(test_dset,bs=512)
    
    
    learn = Learner(None, model.to(def_device), loss_func=1)
    learn.load(model_pth)
    
    learn.model.eval()
    
    preds = []
    for data in test_dl:
        inputs = data.to(def_device)
        outputs = learn.model(inputs) #learn.model(x).sigmoid().detach().cpu().numpy()

        preds.append(outputs.detach().cpu().numpy())

    preds = np.concatenate(preds)
    preds = pd.DataFrame(preds,index=df.index,columns=target_col)

    return preds

In [ ]:
test = df.loc[split0[1]]

In [ ]:
pred = predict_dl(test.head(3),
                  feat_col,
                  target_col, 
                  model,'test')
pred

,-5P,-5G,-5A,-5C,-5S,-5T,-5V,-5I,-5L,-5M,-5F,-5Y,-5W,-5H,-5K,-5R,-5Q,-5N,-5D,-5E,-5s,-5t,-5y,-4P,-4G,-4A,-4C,-4S,-4T,-4V,-4I,-4L,-4M,-4F,-4Y,-4W,-4H,-4K,-4R,-4Q,-4N,-4D,-4E,-4s,-4t,-4y,-3P,-3G,-3A,-3C,-3S,-3T,-3V,-3I,-3L,-3M,-3F,-3Y,-3W,-3H,-3K,-3R,-3Q,-3N,-3D,-3E,-3s,-3t,-3y,-2P,-2G,-2A,-2C,-2S,-2T,-2V,-2I,-2L,-2M,-2F,-2Y,-2W,-2H,-2K,-2R,-2Q,-2N,-2D,-2E,-2s,-2t,-2y,-1P,-1G,-1A,-1C,-1S,-1T,-1V,-1I,-1L,-1M,-1F,-1Y,-1W,-1H,-1K,-1R,-1Q,-1N,-1D,-1E,-1s,-1t,-1y,1P,1G,1A,1C,1S,1T,1V,1I,1L,1M,1F,1Y,1W,1H,1K,1R,1Q,1N,1D,1E,1s,1t,1y,2P,2G,2A,2C,2S,2T,2V,2I,2L,2M,2F,2Y,2W,2H,2K,2R,2Q,2N,2D,2E,2s,2t,2y,3P,3G,3A,3C,3S,3T,3V,3I,3L,3M,3F,3Y,3W,3H,3K,3R,3Q,3N,3D,3E,3s,3t,3y,4P,4G,4A,4C,4S,4T,4V,4I,4L,4M,4F,4Y,4W,4H,4K,4R,4Q,4N,4D,4E,4s,4t,4y,0s,0t,0y
3,-1.124083,-1.333018,0.370999,-1.482608,1.065671,2.435333,-1.181065,0.243360,0.541074,0.526481,0.789112,-1.935882,1.474911,0.908850,-2.656013,0.462956,-0.006826,1.895988,0.339413,0.476922,-0.983320,1.270476,1.475200,-1.669044,-1.078790,-1.765836,0.821250,0.570236,-0.302717,1.042980,-1.587289,0.182884,2.623421,-2.359888,-0.646681,1.590253,2.212846,-2.315615,-0.418003,1.428890,0.929068,1.008114,-0.425482,-2.041824,2.350778,-0.230695,0.340297,0.432018,0.793356,1.710751,-0.199124,0.316310,-2.101466,-1.823912,-1.321935,-1.437125,0.130883,2.243388,1.785522,-1.228163,-1.860953,-2.221823,-0.398452,-1.200318,1.220733,-0.437580,-1.395456,-1.520077,0.929825,-0.452956,-2.240156,1.864775,2.193667,-0.092991,-0.991873,-0.043211,-0.026852,-1.612026,0.882562,-1.191970,0.247461,-1.808025,0.248498,-2.131180,-2.775856,-2.635972,-0.260469,1.875052,-0.374094,-1.802452,1.710730,0.500554,-2.568784,-1.137813,1.847648,-1.176928,-0.784231,-0.335855,-0.998383,-1.808661,0.702443,-0.933349,-1.970836,1.824810,-0.550751,-1.536322,-0.277769,-0.246322,0.715673,-0.243507,-0.004827,0.491623,-0.027491,-0.621791,1.520924,-1.725467,-0.762519,0.823468,-0.832005,-1.385254,1.463969,-1.065097,-0.268777,0.191179,-0.256498,1.285461,1.410629,-0.480923,0.052641,-1.333732,-0.808122,-0.641196,0.846606,0.054078,-2.322969,-1.790822,0.040727,0.311768,2.213273,1.011133,1.735011,2.706975,2.267068,1.176308,-1.155246,0.358741,-0.040418,0.549987,1.847084,-0.215823,-0.858711,-1.608082,-1.274341,-0.575624,0.875942,0.312031,-0.394171,-1.378174,-1.457255,0.548059,-0.840706,2.288101,0.837432,0.640592,1.597600,2.678149,2.460815,-3.140487,2.607033,0.160671,1.281283,0.391561,0.285112,0.313980,1.758898,-0.577085,-1.576527,-0.883964,-0.221910,-0.472430,0.627727,1.816311,-1.552552,-2.032012,1.988275,1.918780,-1.275063,-1.375782,0.033672,-1.072162,-0.100851,-0.315402,-0.936294,-0.149866,1.335872,1.107367,0.755761,0.614412,-0.339984,1.296790,0.086605,-0.325200,0.626342,0.139817,-1.742222,0.486119,0.835788,1.808517,-0.691114,1.019787
8,-1.082794,-1.315036,0.377679,-1.303769,1.028240,2.277051,-1.140310,0.201248,0.481489,0.407088,0.760801,-1.863295,1.475667,0.822474,-2.524725,0.393658,-0.127901,1.751877,0.297822,0.382287,-0.939208,1.297959,1.379947,-1.500982,-1.073636,-1.720251,0.947168,0.489036,-0.398783,1.048175,-1.578853,0.202615,2.475548,-2.330920,-0.625377,1.575310,2.092734,-2.183469,-0.456739,1.414051,0.921833,0.912542,-0.522701,-1.903706,2.279727,-0.167850,0.331005,0.305503,0.690275,1.713022,-0.114235,0.247336,-1.971480,-1.755766,-1.216914,-1.356737,0.120693,2.188673,1.620649,-1.219324,-1.883983,-2.129232,-0.349166,-1.165124,1.202160,-0.446142,-1.265840,-1.397379,0.963779,-0.340953,-2.203007,1.697120,2.142285,-0.070298,-0.790852,-0.047399,-0.163245,-1.561266,0.780174,-1.106945,0.288282,-1.763729,0.198121,-1.961371,-2.751048,-2.657668,-0.304017,1.781997,-0.442557,-1.629253,1.846559,0.596902,-2.496807,-1.063768,1.810785,-1.048127,-0.712356,-0.200570,-0.918905,-1.857498,0.661085,-0.874454,-1.888536,1.837306,-0.410973,-1.405960,-0.243723,-0.118655,0.758936,-0.238423,0.035166,0.525149,0.007085,-0.653448,1.576913,-1.723523,-0.764785,0.659238,-0.681803,-1.357008,1.411377,-1.068875,-0.276262,0.145424,-0.154642,1.320319,1.450584,-0.400749,0.072105,-1.270154,-0.710633,

In [ ]:
_,_,corr = score_each(test[target_col].head(3),pred)

overall MSE: 3.1266
Average Pearson: 0.1593 


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()